<h2>Automated Motion Recorder<h2>


In [8]:
import cv2
import numpy as np
import os
import time
from datetime import datetime

Background Subtraction


In [9]:
# Initialize the video capture
video1 = cv2.VideoCapture(0)

# Directory to save the output video
save_dir = "motion_recordings"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Initialize variables for video recording
fourcc = cv2.VideoWriter_fourcc(*"XVID")
unix_time = time.time()

# Convert the Unix timestamp to a datetime object
date_time = datetime.fromtimestamp(unix_time)

# Format the datetime object as dd-mm-yyyy_HH-MM-SS
formatted_date_time = date_time.strftime("%d-%m-%Y_%H-%M-%S")
filename = os.path.join(save_dir, f"Web Cam{formatted_date_time}.avi")
out = None


# Initialize the background subtractor
backgroundObject = cv2.createBackgroundSubtractorMOG2(
    varThreshold=30, detectShadows=True, history=70
)

# Kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))


while True:
    # Read a new frame
    ret, frame = video1.read()
    if not ret:
        break

    # Apply the background subtractor
    fgmask = backgroundObject.apply(frame)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    fgmask = cv2.dilate(fgmask, kernel, iterations=2)

    # Detect contours in the frame
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If motion is detected, process the frame
    for contour in contours:
        # Ignore small contours to avoid noise
        if cv2.contourArea(contour) < 1000:
            continue

        # Get the bounding box coordinates for the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Draw a green rectangle around the motion area
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

    # If motion is detected, add timestamp and save the video
    if len(contours) > 0:
        # Add timestamp to the frame
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cv2.putText(
            frame,
            f"Motion detected: {timestamp}",
            (10, 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.75,
            (255, 0, 0),
            2,
        )

        # Initialize the video writer if not already started
        if out is None:
            out = cv2.VideoWriter(
                filename, fourcc, 30.0, (frame.shape[1], frame.shape[0])
            )
            print(f"Started saving motion-detected video: {filename}")

        # Write the motion-detected frame to the video
        out.write(frame)

    # Resize both frames to equal dimensions for stacking
    frame_resized = cv2.resize(frame, (640, 480))
    fgmask_resized = cv2.resize(fgmask, (640, 480))

    # Convert `fgmask` to a 3-channel image for stacking
    fgmask_color = cv2.cvtColor(fgmask_resized, cv2.COLOR_GRAY2BGR)

    # Stack the original frame and the foreground mask side by side
    stacked_frames = np.hstack((frame_resized, fgmask_color))

    # Display the stacked frames
    cv2.imshow("Motion Detection (Left: Frame, Right: Mask)", stacked_frames)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
video1.release()
if out:
    out.release()
cv2.destroyAllWindows()

print(f"Motion-detected video saved at: {filename}")

Started saving motion-detected video: motion_recordings\Web Cam12-12-2024_14-50-04.avi
Motion-detected video saved at: motion_recordings\Web Cam12-12-2024_14-50-04.avi


Demonstration using External Camera


In [10]:
# Initialize the video capture
# http://100.83.244.243:8080/


video1 = cv2.VideoCapture("http://100.83.244.243:8080/video")

# Directory to save the output video

save_dir = "motion_recordings"



if not os.path.exists(save_dir):



    os.makedirs(save_dir)



# Initialize variables for video recording



fourcc = cv2.VideoWriter_fourcc(*"XVID")



unix_time = time.time()



# Convert the Unix timestamp to a datetime object



date_time = datetime.fromtimestamp(unix_time)



# Format the datetime object as dd-mm-yyyy_HH-MM-SS



formatted_date_time = date_time.strftime("%d-%m-%Y_%H-%M-%S")



filename = os.path.join(save_dir, f"External Camera{formatted_date_time}.avi")



out = None



# Initialize the background subtractor



backgroundObject = cv2.createBackgroundSubtractorMOG2(



    varThreshold=100, detectShadows=True, history=60
)



# Kernel for morphological operations



kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))



while True:



    # Read a new frame



    ret, frame = video1.read()



    if not ret:



        break




    # Apply the background subtractor



    fgmask = backgroundObject.apply(frame)



    fgmask = cv2.erode(fgmask, kernel, iterations=2)



    fgmask = cv2.dilate(fgmask, kernel, iterations=2)




    # Detect contours in the frame



    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)




    # If motion is detected, process the frame



    for contour in contours:



        # Ignore small contours to avoid noise



        if cv2.contourArea(contour) < 2000:
            continue




        # Get the bounding box coordinates for the contour



        x, y, w, h = cv2.boundingRect(contour)




        # Draw a green rectangle around the motion area



        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)




    # If motion is detected, add timestamp and save the video



    if len(contours) > 0:



        # Add timestamp to the frame



        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")



        cv2.putText(
            frame,



            f"Motion detected: {timestamp}",



            (10, 20),



            cv2.FONT_HERSHEY_SIMPLEX,



            0.75,



            (255, 0, 0),

            2,
        )




        # Initialize the video writer if not already started



        if out is None:



            out = cv2.VideoWriter(



                filename, fourcc, 20.0, (frame.shape[1], frame.shape[0])
            )



            print(f"Started saving motion-detected video: {filename}")




        # Write the motion-detected frame to the video



        out.write(frame)




    # Resize both frames to equal dimensions for stacking



    frame_resized = cv2.resize(frame, (640, 480))



    fgmask_resized = cv2.resize(fgmask, (640, 480))




    # Convert `fgmask` to a 3-channel image for stacking



    fgmask_color = cv2.cvtColor(fgmask_resized, cv2.COLOR_GRAY2BGR)




    # Stack the original frame and the foreground mask side by side



    stacked_frames = np.hstack((frame_resized, fgmask_color))




    # Display the stacked frames



    cv2.imshow("Motion Detection on Remote Camera)", frame_resized)




    # Press 'q' to exit



    if cv2.waitKey(1) & 0xFF == ord("q"):



        break



# Release resources



video1.release()



if out:
    out.release()



cv2.destroyAllWindows()



print(f"Motion-detected video saved at: {filename}")

Started saving motion-detected video: motion_recordings\External Camera12-12-2024_14-50-52.avi
Motion-detected video saved at: motion_recordings\External Camera12-12-2024_14-50-52.avi
